In [1]:
from BackgammonModel import *

In [2]:
model = BackgammonModel( \
    input_size = 198,
    hidden_size = 100,
    output_size = 1,
    trace_decay = 0,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/100/checkpoint_237000.pt',
    eval_interval = 100
)

Loading saved model: ./checkpoints/100/checkpoint_237000.pt


In [3]:
model2 = BackgammonModel( \
    input_size = 198,
    hidden_size = 100,
    output_size = 1,
    trace_decay = 0,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/100/checkpoint_89000.pt',
    eval_interval = 1000
)

Loading saved model: ./checkpoints/100/checkpoint_89000.pt


In [4]:
env = Env()
model.evaluate(env, [RandomAgent(player = 0), 
                     RandomAgent(player = 1)])

100%|██████████| 100/100 [00:05<00:00, 19.03it/s]


{0: 46, 1: 54, <Player.EMPTY: 2>: 0}

In [18]:
env = Env()
model.evaluate(env, [TDAgent(player = 0, net = model2, env = env),
                     TDAgent(player = 1, net = model2, env = env)])

100%|██████████| 100/100 [00:17<00:00,  5.56it/s]


{0: 50, 1: 50, <Player.EMPTY: 2>: 0}

In [5]:
env = Env()
model.evaluate(env, [TDAgent(player = 0, net = model2, env = env),
                     RandomAgent(player = 1),
                     ])

100%|██████████| 100/100 [00:09<00:00, 10.51it/s]


{0: 88, 1: 12, <Player.EMPTY: 2>: 0}

In [6]:
env = Env()
model.evaluate(env, [RandomAgent(player = 0),
                     TDAgent(player = 1, net = model2, env = env)])

100%|██████████| 100/100 [00:07<00:00, 12.94it/s]


{0: 12, 1: 88, <Player.EMPTY: 2>: 0}

In [7]:
env = Env()
model.evaluate(env, [TDAgent(player = 0, net = model, env = env),
                     RandomAgent(player = 1),
                     ])

100%|██████████| 100/100 [00:07<00:00, 13.49it/s]


{0: 58, 1: 42, <Player.EMPTY: 2>: 0}

In [8]:
env = Env()
model.evaluate(env, [RandomAgent(player = 0),
                     TDAgent(player = 1, net = model, env = env)])

100%|██████████| 100/100 [00:07<00:00, 13.21it/s]


{0: 48, 1: 52, <Player.EMPTY: 2>: 0}

In [9]:
env = Env()
model.evaluate(env, [TDAgent(player = 0, net = model2, env = env),
                     TDAgent(player = 1, net = model, env = env)])

100%|██████████| 100/100 [00:12<00:00,  7.85it/s]


{0: 66, 1: 34, <Player.EMPTY: 2>: 0}

In [10]:
env = Env()
model.evaluate(env, [TDAgent(player = 0, net = model, env = env),
                     TDAgent(player = 1, net = model2, env = env)])

100%|██████████| 100/100 [00:10<00:00,  9.18it/s]


{0: 19, 1: 81, <Player.EMPTY: 2>: 0}

In [11]:
%load_ext line_profiler
model = BackgammonModel( \
    input_size = 198,
    hidden_size = 50,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/50/checkpoint_358000.pt',
    eval_interval = 100
)
env = Env()
agents = [RandomAgent(player = 0), TDAgent(player = 1, net = model, env = env)]

Loading saved model: ./checkpoints/50/checkpoint_358000.pt


In [12]:
p, _, _, _ = env.reset()
agent = agents[p]
legal_actions = env.get_legal_actions()
print(legal_actions)
game = env.game

[621, 335, 205, 153, 1297, 1011, 881, 829, 610, 324, 220, 194, 142, 1286, 1000, 922, 870, 818, 605, 319, 189, 137, 1281, 995, 865, 813, 603, 317, 187, 135, 1279, 993, 863, 811]


In [13]:
%lprun -f game.observation_tensor game.observation_tensor(0)

In [14]:
%lprun -f game.step game.step(legal_actions[0])

In [15]:
%lprun -f agent.choose_best_action agent.choose_best_action(legal_actions)

AssertionError: 

In [ ]:
%lprun -f model.evaluate model.evaluate(env, agents, num_episodes = 1)


In [ ]:
from Backgammon import *
game = Backgammon(verbose=False, max_turns = 200)


In [ ]:
%lprun -f game.get_legal_actions game.get_legal_actions()

In [ ]:
%lprun -f game.legal_checker_moves_outer game.legal_checker_moves_outer(0, game.dice)

In [ ]:
%lprun -f game.legal_checker_moves game.legal_checker_moves(game.cur_player, game.dice)

In [ ]:
a = game.legal_checker_moves(game.cur_player, game.dice)[0]
a

In [ ]:
%lprun -f game.apply_move game.apply_move(a)

In [ ]:
%lprun -f game.update_board_pos game.update_board_pos(15, 1, 1)

In [ ]:
env = Env()
env.reset()
legal_actions = env.get_legal_actions()
print(legal_actions)

In [ ]:
def choose_best_action(actions):
    obs_ = []
    tots = []
    for a in actions:
        _, _, obs, _, _ = env.step(a)
        _ = env.step_back(a)
        tot = 0
        for i in range(len(obs)):
            tot +=  obs[i]*i
        tots.append(tot)
    return tots
tots_no_pool = choose_best_action(legal_actions)
print(tots_no_pool)

In [ ]:
from multiprocessing import Pool

def g(a):
    _, _, obs, _, _ = env.step(a)
    _ = env.step_back(a)
    tot = 0
    for i in range(len(obs)):
        tot +=  obs[i]*i
    return tot
def f(x):
    return x*x

with Pool(5) as p:
    tots_pool = p.map(g, legal_actions)
print(tots_pool)

In [ ]:
%%timeit
_ = choose_best_action(legal_actions)


In [ ]:
%%timeit
with Pool(5) as p:
    _ = p.map(g, legal_actions)

In [ ]:
p = Pool(4)

In [ ]:
%%timeit
_ = p.map(g, legal_actions)